##GRID SEARCH USING NEURAL NETWORKS FOR KERAS AND TENSORFLOW

This notebook shows how to implement grid search for parameter tuning on Neural Networks. It makes use of the Grid Search Function in Scikit Learn and also the Keras Classifier  Wrapper. It is implemented using Keras and Tensorflow as its back end

The data set used for demonstration would be the MNIST dataset

##MAKE IMPORTS

In [0]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV as gs

##DEFINE CONSTANTS

In [0]:
batch_size = 128
num_classes = 10
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28


##LOAD DATASET AND SPLIT

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

##Some Preprocessing

In [23]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


#This is where the magic happens, We define a searcher function and its parameters are the parameters we want to tune

In [0]:
def searcher(activation = 'relu', optimizer = 'adam',init_mode = 'uniform',drop_out=0.25):
    # Must define the input shape in the first layer of the neural network
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3),
                     activation=activation,
                     kernel_initializer = init_mode,
                     input_shape=(28,28,1)))
    model.add(Conv2D(64, (3, 3), activation=activation, kernel_initializer = init_mode))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop_out))
    model.add(Flatten())
    model.add(Dense(256, activation=activation,kernel_initializer = init_mode))
    model.add(Dropout(2*drop_out))
    model.add(Dense(10, activation='softmax',kernel_initializer = init_mode))
    
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

##Wrap the model using the keras wrapper

In [0]:
model = KerasClassifier(build_fn=searcher,verbose=1)

##Define the paramters we want to tune and give them estimated values

In [0]:
#Params to tune
activation = ['relu','tanh']
batch = [10,32,64,128,256]
epoch = [1,5,10,20,30]
optimizer = ['SGD','RMSprop','Adagrad','Adadelta','Adam','Adamax']
init = ['uniform','lecun_uniform','he_uniform']
dropout = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [0]:
param = dict(activation = activation, epochs = epoch,batch_size = batch,
             optimizer = optimizer, init_mode = init,drop_out = dropout)

##Add checkpointing to keep track of the best model

In [0]:
from keras.callbacks import ModelCheckpoint
optimum_weight = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)


##Train

In [0]:
grid = gs(estimator=model,param_grid=param, cv = 10,refit=True)

In [0]:
grid_result = grid.fit(x_train,y_train)

##Show the  best result and its parameters

In [0]:
print('Best: ',grid_result.best_score_, 'Params: ', grid_result.best_params_)